# Ray Secrets Proxy

The Ray Secrets Proxy offers a Ray native approach to managing secrets in a Ray or Anyscale environment.  This walkthrough will show how to use the Ray Secrets Proxy for either the AWS Secrets Manager or GCP Secrets Manager.

## Initializing the environment and import classes

In [ ]:
import ray
from ray_secret_operator import AWSRaySecretOperator, GCPRaySecretOperator
from ray_secret_proxy import RaySecretProxy

ray.init()

## AWS Secrets Manager

In [ ]:
# Initialize the AWS Operator
operator = AWSRaySecretOperator()


This can also be done by explicitly passing credentials into the operator
```
operator = AWSRaySecretOperator(
    aws_access_key_id=ABC,
    aws_secret_access_key=ABC,
    aws_session_token=XYZ
)
```

## Sample GCP code
```
project_id = 'ABCXYZ'

# Initialize the GCP Operator
gcp_op = GCPRaySecretOperator(project_id=project_id) 
```

You can also pass in an explicit credentials dictionary to the operator
```
your_gcp_credential_dict= { /credential material/ }
operator = GCPRaySecretOperator(project_id=project_id, credentials=your_gcp_credential_dict) 
```

In [ ]:

# Create the Secret Proxy with a default TTL of -1
ssm_proxy = RaySecretProxy.remote(ray_secret_operator=operator) 

TTL of -1 means the secret will never expire and fetched secrets will be stored in the proxy cache

You can also pass in other TTLs:

TTL = 0 means no secret will be cached in the proxy.  Any secrets will be fetched at runtime.  
However, by expicitly fetching a secret, you can override the expiry time of the secret that is returned.  It will not be persisted in the cache.
```
aws_ssm_proxy = RaySecretProxy.remote(aws_op, ttl=0)
```

TTL > 0 means the secret, once fetched, will remain cached and valid for X seconds. 
If the secret is expired when requested, it will be fetched again upon request
```
aws_ssm_proxy = RaySecretProxy.remote(aws_op, ttl=3600)
```

## List secrets 

In [ ]:

list_of_secrets = ray.get(ssm_proxy.list_secrets.remote())


Returns a list of names that are stored in the SSM for the AWS account/region or GCP project

You can pass in a filter object based on the filters outlined in the AWS/GCP documentation:
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/secretsmanager.html#SecretsManager.Client.list_secrets
https://cloud.google.com/secret-manager/docs/reference/rest/v1/projects.secrets/list

AWS example:
```
list_of_secrets = ray.get(ssm_proxy.list_secrets.remote(
    filter=[
        {
            'Key': 'tag-key',
            'Values': [
                'Dev','Ray','Anyscale
            ]
        },
    ]
))
```
GCP example:
```
list_of_secrets = ray.get(ssm_proxy.list_secrets.remote(
    filter="name:Dev OR name:Ray OR name:Anyscale"
))
```


## Hydrate the cache with all available secrets

In [ ]:

ray.get(ssm_proxy.hydrate_cache.remote())


This will fetch all secrets available from the list_secrets operation and store them into the proxy cache (only if the TTL is not 0)

Like list_secrets, you can pass in the filter argument.

In addition, you can pass in an argument for secret_names, a list of secret names to pull into the cache/
```
ray.get(ssm_proxy.hydrate_cache.remote(
    secret_names = ['supersecret1', 'supersecret2']
))
```

## Removing Secrets from the Cache


In [ ]:

# Listing available secrets in the cache
ray.get(ssm_proxy.list_cached_secrets.remote())
'''
Returns a list of secret names that are stored in the cache

'''

# Purge a specific secret from the cache
ray.get(ssm_proxy.purge_secret.remote('supersecret1'))
'''
If the secret is stored in the secret cache, it will be removed.  New attempts to fetch the secret will try and fetch from the SSM.
'''

# Purge all secrets from cache
ray.get(ssm_proxy.purge_cache.remote())


## Retrieving secrets

In [ ]:
# Retrieve a specific secret
secret = ray.get(ssm_proxy.get_secret.remote(secret_name='supersecret'))

This will attempt to retrieve the Ray Secret named 'supersecret' from either the AWS or GCP secret manager.

### Adding a TTL
You can add a TTL to a specific secret upon retrieval.  The TTL value meanings are the same as the Ray Secret Proxy.

However, the TTL passed to get_secret is persisted to the secret itself, changing the behavior of the cache hit/refresh and providing expiry information to the consumer if no secrets are cached in the proxy.
```
# Cache the secret if Proxy TTL is not 0
secret = ray.get(ssm_proxy.get_secret.remote(secret_name='supersecret', ttl=-1))

# Cache the secret for X seconds if Proxy TTL is not 0
secret = ray.get(ssm_proxy.get_secret.remote(secret_name='supersecret', ttl=3600))

# Do not cache the Secret
secret = ray.get(ssm_proxy.get_secret.remote(secret_name='supersecret', ttl=0))
```


### AWS Variants 
```
secret = ray.get(ssm_proxy.get_secret.remote(secret_name='<SECRET_ARN_OR_NAME>'))
secret = ray.get(ssm_proxy.get_secret.remote(secret_name='<SECRET_ARN_OR_NAME>', VersionState='string'))
secret = ray.get(ssm_proxy.get_secret.remote(secret_name='<SECRET_ARN_OR_NAME>', VersionId='string'))
secret = ray.get(ssm_proxy.get_secret.remote(secret_name='<SECRET_ARN_OR_NAME>', VersionId='string', VersionState='string'))
```

### GCP Variants
```
secret = ray.get(ssm_proxy.get_secret.remote(secret_name='<SECRET_ARN>'))
secret = ray.get(ssm_proxy.get_secret.remote(secret_name='<SECRET_NAME>/versions/<VERSION>'))
secret = ray.get(ssm_proxy.get_secret.remote(secret_name='projects/<PROJECT_ID>/secrets/<SECRET_NAME>'))
secret = ray.get(ssm_proxy.get_secret.remote(secret_name='projects/<PROJECT_ID>/secrets/<SECRET_NAME>/versions/<VERSION>'))
```

## Accessing a Secret
The Ray Secret is a class that does not immediately expose secret material.  To access the secret, use the secret.value() method.

In [ ]:
print(secret) #return SECRET_NAME: ********
print(secret.value()) # returns SECRET_VALUE < -- Don't do this

print(secret.metadata) #returns any extra information that was returned as part of the get_secret operation
print(secret.is_expired()) #returns a boolean True if the secret is still valid based on it's TTL, False if otherwise